<a href="https://colab.research.google.com/github/lightsixer/P1regression/blob/dev/notebooks/stackingModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
#Group 13 Stacking Model
#partly based on information from this link https://medium.com/@brijesh_soni/stacking-to-improve-model-performance-a-comprehensive-guide-on-ensemble-learning-in-python-9ed53c93ce28

import pickle #importing libraries
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd

pickleFile = open("../models/rforest.pkl", "rb") #opening pickle files containing the models
pickleFile2 = open("../models/knnModel.pkl", "rb")
pickleFile3 = open("../models/gboostModel.pkl", "rb")
pickleFile4 = open("../models/xgboost.pkl", "rb")

rfModel = pd.read_pickle(pickleFile) #placing the models into variables
knnModel = pd.read_pickle(pickleFile2)
gboostModel = pd.read_pickle(pickleFile3)
xgboostModel = pd.read_pickle(pickleFile4)

location = '../data/preptrain.csv' #obtaining location of data file
data = pd.read_csv(location)
data.head()

X = data.drop('SalePrice', axis = 1) #dropping target variable from data
y = data.SalePrice #y is the target
#data is split along 80 for training and 20 for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8945, shuffle = True)
#random forest regression method makes prediction
rfrPredict = rfModel.predict(X_test)
r2Result = r2_score(y_test,rfrPredict)
print("r2 score for Random Forest Regressor is : ", r2Result)
#knn regression method makes prediction
knnPredict = knnModel.predict(X_test)
r2Result = r2_score(y_test,knnPredict)
print("r2 score for KNN Regressor is : ", r2Result)
#gradient boost regression method makes prediction
gboostPredict = gboostModel.predict(X_test)
r2Result = r2_score(y_test,knnPredict)
print("r2 score for gradient boosting regressor is : ", r2Result)
#XGBoost regressor method makes prediction
xgbPredict = xgboostModel.predict(X_test)
r2Result = r2_score(y_test,xgbPredict)
print("r2 score for XGBoost regressor is : ", r2Result)
print("")
#predictions are used for stacking model
stacker = np.column_stack((rfrPredict,knnPredict, gboostPredict, xgbPredict))
stackingModel = LinearRegression()
bestR = stackingModel.fit(stacker, y_test) #stack is used for training
predicter = stackingModel.predict(stacker)
r2Result = r2_score(y_test,predicter)
print("r2 score for the stacking model is: ", r2Result)

errors = abs(predicter - y_test) #testing for accuracy and error
mape = 100 * np.mean(errors/ y_test)
accuracy = 100 - mape
print('Average Error: {:0.4f} dollars.'.format(np.mean(errors)))
print('Accuracy = {:0.2f}%.'.format(accuracy))

import pickle
import os
pickle.dump(bestR, open(os.path.join('../models/', 'stackingModel.pkl'), 'wb'), protocol=4)

r2 score for Random Forest Regressor is :  0.879236372614344
r2 score for KNN Regressor is :  0.6099639364310201
r2 score for gradient boosting regressor is :  0.6099639364310201
r2 score for XGBoost regressor is :  0.9477147371002627

r2 score for the stacking model is:  0.9517387529487127
Average Error: 9687.5148 dollars.
Accuracy = 94.12%.
